In [ ]:
# pip install "generative-ai-hub-sdk[all]"

In [2]:
import os

# please enter the Credentials from your AI core landscape.
env_vars = {
    'AICORE_AUTH_URL' : 'https://ai-dev-2025-jypl7lq7.authentication.us21.hana.ondemand.com',
    'AICORE_CLIENT_ID' : 'sb-cb3e467a-adbc-4d9d-ae6b-a4c62441c6d7!b33974|xsuaa_std!b22746',
    'AICORE_CLIENT_SECRET' : '5339fb21-1cc8-45cf-996a-ee87982f66dc$wYOmtC9o6S9ypdCwwiJR4Cr_uKysaO9c9sl_xvWLhfk=',
    'AICORE_BASE_URL' : 'https://api.ai.prod-us21.eastus.azure.ml.hana.ondemand.com/v2',
    'AICORE_RESOURCE_GROUP' : 'new-resource-group'
}

# Set the environment variables using `os.environ`.
for key, value in env_vars.items():
    os.environ[key] = value

In [1]:
# Read data from 'GRAPH_DOCU_2503.csv' and store each row in the 'data' list
import csv

data = []
with open('GRAPH_DOCU_2503.csv', encoding='utf-8') as csvfile:
    csv_reader = csv.reader(csvfile)
    for row in csv_reader:
        try:
            data.append(row)
        except:
            print(row)

print(data[0])  # Print the first row of data

['ID', 'L1', 'L2', 'L3', 'FILENAME', 'HEADER1', 'HEADER2', 'TEXT', 'VECTOR_STR']


In [ ]:
# Establish a secure connection to an SAP HANA database using hdbcli 
# pip install hdbcli
import hdbcli
from hdbcli import dbapi

cc = dbapi.connect(
    address='7a1c2ce7-c54f-4137-b298-f7e93e1f50e5.hana.prod-us21.hanacloud.ondemand.com',
    port='443',
    user='DEVELOPER',
    password='Bticto25!',
    encrypt=True
    )


In [48]:
# Create a table
cursor = cc.cursor()
sql_command = '''CREATE TABLE TABLENAME_AI(ID1 BIGINT, ID2 BIGINT, L1 NVARCHAR(3), L2 NVARCHAR(3), L3 NVARCHAR(3), FILENAME NVARCHAR(100), HEADER1 NVARCHAR(5000), HEADER2 NVARCHAR(5000), TEXT NCLOB, VECTOR_STR REAL_VECTOR);'''
cursor.execute(sql_command)
cursor.close()


ProgrammingError: (288, 'cannot use duplicate table name: TABLENAME_AI: line 1 col 14 (at pos 13)')

In [ ]:
# Inserting data into the specified table using a prepared SQL statement with real vector conversion.
cursor = cc.cursor()
sql_insert = 'INSERT INTO TABLENAME_AI(ID1, ID2, L1, L2, L3, FILENAME, HEADER1, HEADER2, TEXT, VECTOR_STR) VALUES (?,?,?,?,?,?,?,?,?,TO_REAL_VECTOR(?))'
cursor.executemany(sql_insert,data[1:])


In [ ]:
# pip install hana_ml
import hana_ml
print(hana_ml.__version__)
from gen_ai_hub.proxy.core.proxy_clients import get_proxy_client
proxy_client = get_proxy_client('gen-ai-hub') # for an AI Core proxy

print(proxy_client)

2.24.25040300
base_url=None auth_url=None client_id=None client_secret=None resource_group=None ai_core_client=<ai_core_sdk.ai_core_v2_client.AICoreV2Client object at 0x169125a90>


In [8]:
# Get embeddings
from gen_ai_hub.proxy.native.openai import embeddings

def get_embedding(input, model="text-embedding-ada-002") -> str:
    response = embeddings.create(
      model_name=model,
      input=input
    )
    return response.data[0].embedding


In [21]:
# Perform a vector search on the table using the specified metric and return the top k results
cursor = cc.cursor()
def run_vector_search(query: str, metric="COSINE_SIMILARITY", k=4):
    if metric == 'L2DISTANCE':
        sort = 'ASC'
    else:
        sort = 'DESC'
    query_vector = get_embedding(query)
    sql = '''SELECT TOP {k} "ID2", "TEXT"
        FROM "TABLENAME_AI"
        ORDER BY "{metric}"("VECTOR_STR", TO_REAL_VECTOR('{qv}')) {sort}'''.format(k=k, metric=metric, qv=query_vector, sort=sort)
    cursor.execute(sql)
    hdf = cursor.fetchall()
    return hdf[:k]


In [22]:
# Create a prompt template
promptTemplate_fstring = """
You are an SAP HANA Cloud expert.
You are provided multiple context items that are related to the prompt you have to answer.
Use the following pieces of context to answer the question at the end. 
Context:
{context}
Question:
{query}
"""
from langchain.prompts import PromptTemplate
promptTemplate = PromptTemplate.from_template(promptTemplate_fstring)


In [25]:
# Import necessary modules, and define a function to query an LLM with a formatted prompt and vector-based context
# pip install tiktoken
import tiktoken

from gen_ai_hub.proxy.langchain.openai import ChatOpenAI
print(ChatOpenAI)

def retrieve_and_query_llm(query: str, metric='COSINE_SIMILARITY', k = 4) -> str:
    context = ''
    context = run_vector_search(query, metric, k)
    prompt = promptTemplate.format(query=query, context=' '.join(str(context)))
    encoding = tiktoken.get_encoding("cl100k_base")
    num_tokens = len(encoding.encode(str(prompt)))
    print('no of tokens'+ str(num_tokens))
    llm = ChatOpenAI(proxy_model_name='gpt-4-32k',max_tokens = 8000)
    response = llm.invoke(prompt).content
    print('Query: '+ query)
    print('\nResponse:')
    print(response)


<class 'gen_ai_hub.proxy.langchain.openai.ChatOpenAI'>


In [26]:
# Query the LLM with a request about calculating the shortest path and retrieve the response

query = "I want to calculate a shortest path. How do I do that?"
response = retrieve_and_query_llm(query=query, k=4)
response


ValueError: No deployment found with: deployment.model_name == text-embedding-ada-002